In [96]:
import tiktoken

In [97]:
tokenizer=tiktoken.get_encoding("gpt2")


In [98]:
with open("the-verdict.txt","r") as f:
    raw_text=f.read()

encoded_text=tokenizer.encode(raw_text)
print(len(encoded_text))    



5145


In [99]:
context_size=4

x=encoded_text[:context_size]
y=encoded_text[1:context_size+1]
   
print(x)
print("   ",y)   


[40, 367, 2885, 1464]
    [367, 2885, 1464, 1807]


In [100]:
for i in range(1,context_size+1):
    context=encoded_text[:i]
    desired=encoded_text[i]
    print(context,"===>",desired)

[40] ===> 367
[40, 367] ===> 2885
[40, 367, 2885] ===> 1464
[40, 367, 2885, 1464] ===> 1807


In [101]:
for i in range(1,context_size+1):
    context = encoded_text[:i]  
    desired = encoded_text[i]
    print(tokenizer.decode(context), "===>", tokenizer.decode([desired]))


I ===>  H
I H ===> AD
I HAD ===>  always
I HAD always ===>  thought


Datasets and Data Loader


Datasets and Dataloader allows us to load data in more effificient manner

we will implement Data Loader that Fatches the Data Using sliding window approach   

In [102]:
from torch.utils.data import Dataset, DataLoader
import torch

In [103]:
class GPTversion1(Dataset):
    def __init__(self,text#text file to be read
                 ,tokenizer #byte pair tokenizer
                 ,max_length #cotext size
                 ,stride):
        self.input_ids=[]
        self.target_ids=[]
        
        token_ids=tokenizer.encode(text,allowed_special={'<|endoftext|>'})

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))


    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,i):
        return self.input_ids[i],self.target_ids[i]

In [104]:
#create Dataset loader
def create_dataset_Loader_V1(text,batch_size=4#number of CPU cores to use
                         ,max_length=256,
                         stride=128,
                         shuffle=True,
                         drop_last=True,
                         num_workers=0#number of CPU threads that we can use simultaneously
                         ):
    #initialize the tokenizer
    tokenizer=tiktoken.get_encoding("gpt2")
    dataset=GPTversion1(text,tokenizer,max_length,stride)
    dataLoader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
    return dataLoader





In [105]:
import torch
with open("the-verdict.txt","r") as f:
    raw_text=f.read()
dataloader=create_dataset_Loader_V1(raw_text,batch_size=1,max_length=4,
                                    stride=1)
data_iter=iter(dataloader)
first_batch=next(data_iter)
print(first_batch)




[tensor([[373, 407, 691, 262]]), tensor([[ 407,  691,  262, 9074]])]


Import train model to create Vector Embeddings

In [106]:
word_vectors =model

In [107]:
print(word_vectors["computers"])

[ 0.32421875 -0.24316406  0.11523438  0.25976562 -0.18847656  0.10595703
 -0.10205078  0.10693359  0.28710938  0.01428223  0.0100708  -0.20214844
  0.19238281  0.07714844 -0.03686523  0.06933594 -0.0013504   0.26757812
  0.12011719  0.02746582 -0.0072937  -0.04443359  0.15625     0.10693359
  0.1640625  -0.07177734  0.02355957 -0.03930664 -0.05004883 -0.17480469
 -0.06054688 -0.10839844 -0.17382812  0.01843262  0.14160156 -0.4140625
 -0.43554688 -0.12792969  0.1484375  -0.04882812 -0.11914062  0.23046875
  0.265625    0.10400391  0.27929688  0.06933594 -0.03881836  0.31640625
 -0.40625     0.05712891 -0.01324463 -0.09960938  0.05737305 -0.18945312
 -0.15039062  0.23632812 -0.05102539 -0.17871094 -0.21972656  0.14746094
  0.16308594  0.04736328 -0.13183594  0.22070312 -0.04003906  0.05517578
 -0.2734375   0.42773438 -0.25585938  0.06591797  0.05419922 -0.25
  0.14453125 -0.00531006 -0.08984375 -0.01312256  0.08349609 -0.203125
 -0.0022583  -0.25390625  0.08935547  0.08447266  0.27539062

In [108]:
print(word_vectors["king"].shape)

(300,)


In [109]:
vocab_size=5
out_dim=3

embedding_layer=torch.nn.Embedding(vocab_size,out_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.5293, -0.4622, -2.5534],
        [-1.1552, -1.1755,  0.3494],
        [ 1.1887, -0.5188, -0.1132],
        [ 0.1112,  0.2234, -0.8278],
        [-0.0327,  1.0452,  2.6432]], requires_grad=True)


In [110]:
input=torch.tensor([1,2,3,4])

In [111]:
print(embedding_layer(input))

tensor([[-1.1552, -1.1755,  0.3494],
        [ 1.1887, -0.5188, -0.1132],
        [ 0.1112,  0.2234, -0.8278],
        [-0.0327,  1.0452,  2.6432]], grad_fn=<EmbeddingBackward0>)


Positional Embedding:

position of words in the vectors also matters the most as the occuarance of the word may create a impact on the meaning of the total sentense

example: 
1) the cat sat on the mat   
2) on the mat the cat sat



There are two type os positional Embedding 
1) Absolute
for each position in the input sequence a unique embedding is added to the tokens embedding to convey it's exact location


2) Relative
The emphasis is on the relative position or the distance between the token
the model learns relationships in terms of how far apart rather than at which exact position
Advantage of Relative positional Encoding: Models can generalize better to the sequence of varying length even if it has not seen it before in the training


Both of the types of Embeddings enables LLM's to understand the orders and relationship between the tokens,ensuring most accurate context aware prediction



The choice between the Two Depends on the application and the nature of the data to be processed    
Absolute Positional Embedding: Suitable when fixed order of tokens  is crusial,such as sequence generation

Relative Positional Embedding:It is suitable for task like language modeling over long Sequences ,where same phrase can appear in different part of the Sequence

Open AI's GPT model Uses Positional Embedding that are optimized during the training process. This optimization is thre part of training itself

In [112]:
vector_size=50257
output_dim=256

In [113]:
token_embedding=torch.nn.Embedding(vector_size,output_dim)

In [114]:
max_length=4
downloader=create_dataset_Loader_V1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=True,drop_last=True
                                    )

In [115]:
data_iter=iter(downloader)
input,output=next(data_iter)

In [116]:
print(input.shape)

torch.Size([8, 4])


In [117]:
print(input_embedding)

tensor([[[-0.5556, -1.0137, -0.5938,  ...,  0.5493,  0.5578,  0.0538],
         [ 0.9017,  0.1570, -1.3037,  ..., -0.3306,  1.5625, -1.3916],
         [ 2.0968,  1.2696, -2.2151,  ..., -0.5722, -1.8061, -0.7265],
         [ 0.1302, -2.0691, -0.0416,  ...,  1.9144,  0.7264, -0.1596]],

        [[-0.4814, -1.8321, -0.4672,  ...,  0.5026, -1.1142,  0.6402],
         [ 2.0756, -1.4190,  1.6017,  ..., -2.6732,  0.1429, -2.8014],
         [ 1.6733, -0.5374, -0.5912,  ..., -0.7966, -0.7311,  1.7051],
         [ 0.5936,  0.1109, -0.9556,  ..., -1.2246,  2.1985, -1.0656]],

        [[-0.4814, -1.8321, -0.4672,  ...,  0.5026, -1.1142,  0.6402],
         [ 2.2799,  0.9102, -0.6695,  ...,  0.3464, -1.7040, -2.0921],
         [ 1.8379, -1.9353, -0.5752,  ...,  0.1207, -0.4134, -0.9853],
         [ 0.7345, -1.5084,  0.3634,  ...,  0.9297,  1.6863, -1.6111]],

        ...,

        [[ 0.2214, -1.2381, -0.4062,  ..., -0.0651, -0.0430, -0.5761],
         [ 0.6183, -1.1724,  0.4823,  ..., -1.3439,  0.66

In [118]:
input_embedding=token_embedding(input)

In [119]:
context_size=max_length
pos_embedding_layer=torch.nn.Embedding(context_size,output_dim)

pos_embedding=pos_embedding_layer(torch.arange(context_size))
print(pos_embedding)

tensor([[ 0.6126, -1.1403, -0.5397,  ..., -1.4797,  1.6822, -0.4507],
        [ 0.9127, -0.4203,  0.6507,  ...,  0.1688, -0.2156, -1.2880],
        [-0.0624, -1.1176, -0.2275,  ..., -0.3903, -0.5160,  0.1056],
        [ 3.1515,  2.0879,  1.4351,  ...,  1.2647,  0.2461,  1.7881]],
       grad_fn=<EmbeddingBackward0>)


In [120]:
input_embedding=input_embedding+pos_embedding
print(input_embedding)

tensor([[[-0.7371,  0.2306, -1.6115,  ..., -1.6740,  2.2484, -3.4074],
         [ 0.5750,  0.8643, -0.2037,  ..., -0.7230,  0.9739, -1.8623],
         [ 0.8442, -0.9500, -0.0305,  ..., -1.2798,  0.5857,  0.0661],
         [ 3.3909,  4.3583,  2.8643,  ...,  1.1085,  1.0910,  2.3179]],

        [[-0.5981, -0.6800, -0.0738,  ...,  0.4951,  1.5737, -1.5129],
         [ 0.8636, -0.4133, -0.4366,  ...,  0.9643, -0.1109, -2.4257],
         [-1.2988, -0.9910, -0.2506,  ...,  1.9407,  0.5146, -0.4867],
         [ 1.9151,  2.2145,  1.4120,  ...,  3.5956,  1.2767,  1.1958]],

        [[ 1.6817,  0.8673,  0.5973,  ..., -0.0594, -0.1428, -0.2399],
         [ 1.0779, -1.7758,  1.1519,  ...,  0.5540, -2.1264, -0.1607],
         [-0.6146, -2.1474,  0.9044,  ...,  0.3396, -0.6378,  1.3975],
         [ 1.3510,  2.2466,  1.0229,  ...,  0.8645,  0.0751, -0.5177]],

        ...,

        [[-1.6841, -1.9304,  0.9577,  ..., -1.2288,  2.0470, -1.2654],
         [ 1.2335,  1.2125, -0.0659,  ..., -0.7786, -1.52